In [ ]:
!pip install open3d trimesh

In [2]:
import trimesh
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import torch
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


------------------------------

In [3]:
file = '/kaggle/input/figures/test_task_meshes/0.obj'

In [4]:
mesh1 = o3d.io.read_triangle_mesh(file)
mesh = o3d.t.geometry.TriangleMesh.from_legacy(mesh1)

scene = o3d.t.geometry.RaycastingScene()
scene.add_triangles(mesh)

min_bound = mesh.vertex.positions.min(0).numpy()
max_bound = mesh.vertex.positions.max(0).numpy()

N = 50000
query_points = np.random.uniform(low=min_bound, high=max_bound,
                                size=[N, 3]).astype(np.float32)
signed_distance = scene.compute_signed_distance(query_points).numpy()

In [ ]:
import plotly.graph_objects as go
import numpy as np
import open3d as o3d

mesh1 = o3d.io.read_triangle_mesh(file)
mesh = o3d.t.geometry.TriangleMesh.from_legacy(mesh1)

vertices = np.asarray(mesh1.vertices)
faces = np.asarray(mesh1.triangles)

mesh_trace = go.Mesh3d(
    x=vertices[:, 0],
    y=vertices[:, 1],
    z=vertices[:, 2],
    i=faces[:, 0],
    j=faces[:, 1],
    k=faces[:, 2],
    color='grey',
    opacity=0.8
)

sdf_trace = go.Scatter3d(
    x=query_points[:, 0],
    y=query_points[:, 1],
    z=query_points[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        color=signed_distance,
        colorscale='Viridis',
        opacity=0.8,
    )
)

layout = go.Layout(
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z'),
    )
)

fig = go.Figure(data=[mesh_trace, sdf_trace], layout=layout)

fig.update_layout(title="Mesh and Signed Distance Function (SDF) Visualization")
fig.show()


In [6]:
from sklearn.neural_network import MLPRegressor
regr = MLPRegressor(solver='adam', hidden_layer_sizes=(256, 128))

regr.fit(query_points, signed_distance)

MLPRegressor(hidden_layer_sizes=(256, 128))

In [7]:
regr.predict(query_points)

array([0.01550402, 0.38081723, 0.13111246, ..., 0.20989986, 0.02528971,
       0.08254127], dtype=float32)

In [8]:
regr.score(query_points, signed_distance)

0.9770637557287088

In [9]:
from sklearn.metrics import f1_score

def calculate_occupancy_surface_f1(true_sdf, predicted_sdf, threshold, noise_std):
    true_sdf = true_sdf + np.random.normal(0, noise_std, predicted_sdf.shape)

    true_occupancy = np.where(true_sdf <= threshold, 1, 0)
    predicted_occupancy = np.where(predicted_sdf <= threshold, 1, 0)
    f1 = f1_score(true_occupancy, predicted_occupancy)

    return f1

In [10]:
def calculate_occupancy_box_f1(true_sdf, predicted_sdf, threshold, noise_std, bounding_volume):
    true_sdf = true_sdf + np.random.normal(0, noise_std, predicted_sdf.shape)

    true_occupancy = np.where(true_sdf <= threshold, 1, 0)
    predicted_occupancy = np.where(predicted_sdf <= threshold, 1, 0)
    
    points_within_volume = np.where((bounding_volume[0] <= true_sdf) & (true_sdf <= bounding_volume[1]))[0]
    true_occupancy = true_occupancy[points_within_volume]
    predicted_occupancy = predicted_occupancy[points_within_volume]
    
    f1 = f1_score(true_occupancy, predicted_occupancy)

    return f1

In [12]:
import time
import tqdm

files = [f'/kaggle/input/figures/test_task_meshes/{i}.obj' for i in range(0, 50)]
avg_surf_f1 = []
avg_box_f1 = []
scores = []
times = []

threshold = 0
noise_std = 1e-2

for file in tqdm.tqdm(files):
    mesh1 = o3d.io.read_triangle_mesh(file)
    mesh = o3d.t.geometry.TriangleMesh.from_legacy(mesh1)

    scene = o3d.t.geometry.RaycastingScene()
    scene.add_triangles(mesh)

    min_bound = mesh.vertex.positions.min(0).numpy()
    max_bound = mesh.vertex.positions.max(0).numpy()

    N = 50000
    
    query_points = np.random.uniform(low=min_bound, high=max_bound,
                                    size=[N, 3]).astype(np.float32)
    signed_distance = scene.compute_signed_distance(query_points).numpy()
    
    
    regr = MLPRegressor(solver='adam', hidden_layer_sizes=(256, 128))
    regr.fit(query_points, signed_distance)
    start = time.time_ns()
    prediction = regr.predict(query_points)
    end = time.time_ns()
    times.append(end - start)
    scores.append(regr.score(query_points, signed_distance))

    surf_f1 = calculate_occupancy_surface_f1(signed_distance, prediction, threshold, noise_std)
    avg_surf_f1.append(surf_f1)

    bounding_volume = (min(min_bound), max(max_bound))
    box_f1 = calculate_occupancy_box_f1(signed_distance, prediction, threshold, noise_std, bounding_volume)
    avg_box_f1.append(box_f1)

100%|██████████| 50/50 [20:35<00:00, 24.72s/it]


In [13]:
print('Average F1 scores:', sum(avg_surf_f1) / len(avg_surf_f1), sum(avg_box_f1) / len(avg_box_f1))
print('Average score:', sum(scores) / len(scores))

Average F1 scores: 0.6704711969657179 0.67002113972821
Average score: 0.9337535046530705


In [16]:
print(f'Pediction time for N = 50 000', min(times), 'ns')

Pediction time for N = 50 000 187309683 ns


In [ ]:
import plotly.graph_objects as go
import numpy as np
import open3d as o3d

file = '/kaggle/input/figures/test_task_meshes/0.obj'
mesh1 = o3d.io.read_triangle_mesh(file)
mesh = o3d.t.geometry.TriangleMesh.from_legacy(mesh1)

vertices = np.asarray(mesh1.vertices)
faces = np.asarray(mesh1.triangles)

N = 50000
    
query_points = np.random.uniform(low=min_bound, high=max_bound,
                                size=[N, 3]).astype(np.float32)
signed_distance = scene.compute_signed_distance(query_points).numpy()


regr = MLPRegressor(solver='adam', hidden_layer_sizes=(256, 128))
regr.fit(query_points, signed_distance)
prediction = regr.predict(query_points)

mesh_trace = go.Mesh3d(
    x=vertices[:, 0],
    y=vertices[:, 1],
    z=vertices[:, 2],
    i=faces[:, 0],
    j=faces[:, 1],
    k=faces[:, 2],
    color='grey',
    opacity=0.5
)

sdf_trace = go.Scatter3d(
    x=query_points[:, 0],
    y=query_points[:, 1],
    z=query_points[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        color=prediction,
        colorscale='Viridis',
        opacity=0.8,
    )
)

layout = go.Layout(
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z'),
    )
)

fig = go.Figure(data=[mesh_trace, sdf_trace], layout=layout)

fig.update_layout(title="Mesh and Signed Distance Function (SDF) Visualization")
fig.show()
